In [1]:
import os
from PIL import Image
import numpy as np
from torchvision import transforms

In [26]:
def calculate_mean_std(root_dir):
    transform_to_tensor = transforms.ToTensor()
    num_pixels = 0
    total_sum = np.zeros(3)
    total_sum_squared = np.zeros(3)

    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.png'): # add more conditions if there are other image types
                filepath = os.path.join(dirpath, filename)
                with Image.open(filepath) as img:
                    img = img.convert('RGB')
                    tensor_img = transform_to_tensor(img)
                    total_sum = np.add(total_sum, (tensor_img.sum((1, 2))))  # sum over height and width dimensions
                    total_sum_squared = np.add(total_sum_squared, ((tensor_img ** 2).sum((1, 2))))  # sum over height and width dimensions
                    _, height, width = tensor_img.shape
                    num_pixels += height * width

    mean = total_sum / num_pixels
    std = np.sqrt((total_sum_squared / num_pixels) - (mean ** 2))
    return mean, std

In [27]:
calculate_mean_std('./data')

(tensor([0.5375, 0.4315, 0.3818], dtype=torch.float64),
 tensor([0.2928, 0.2656, 0.2614], dtype=torch.float64))